<a href="https://colab.research.google.com/github/virf96/Basico/blob/main/Multinomial_Support_Vector_Machine.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Support Vector Machine

## Preparación de ambiente

### Carga de módulos

In [ ]:
import numpy as np
import pandas as pd
from PIL import Image
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.model_selection import train_test_split, cross_val_score

### Funciones relevantes

In [ ]:
def get_data(number):
    img = Image.open('sample%d_black_r.png'%(number)).convert('L')
    img_arr = np.array(img)
    WIDTH, HEIGHT = img.size
    data = list(img.getdata())
    data = [data[offset:offset+WIDTH] for offset in range(0, WIDTH*HEIGHT, WIDTH)]
    return data

In [ ]:
def print_data(data):
    for row in data:
        print(''.join('{:3}'.format(value) for value in row))

In [ ]:
def flatten_data(data):
    return[np.reshape(data, (28*28,))]

In [ ]:
get_data(3)

FileNotFoundError: ignored

In [ ]:
flatten_data(get_data(3))

[array([  0,   0,   0,   0,   0,   0,   1,   4,   3,   0,   0,   0,   0,
          0,   0,   0,   0,   4,   1,   0,   0,   0,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   2,   3,   0,   0,   0,   9,  44,
         79,  99,  97,  73,  35,   0,   0,   0,   2,   0,   0,   0,   0,
          0,   0,   0,   0,   0,   0,   0,   1,   0,   0,  24,  93, 162,
        217, 249, 255, 255, 255, 255, 245, 199,  99,   2,   0,   2,   0,
          0,   0,   0,   0,   0,   0,   0,   0,   1,   0,  30, 140, 227,
        255, 255, 255, 255, 255, 248, 253, 255, 255, 255, 255, 192,  36,
          0,   2,   0,   0,   0,   0,   0,   0,   0,   1,   0,  57, 223,
        255, 255, 252, 207, 154, 101,  60,  40,  51,  95, 185, 255, 250,
        255, 224,  30,   1,   2,   0,   0,   0,   0,   0,   0,   1,   0,
         77, 218, 231, 162,  60,   6,   0,   0,   0,   0,   0,   0,   0,
        110, 249, 247, 255, 144,   0,   3,   0,   0,   0,   0,   0,   0,
          0,   0,   2,  32,  34,   0,   0,   0,   3

In [ ]:
print_data(get_data(3))

  0  0  0  0  0  0  1  4  3  0  0  0  0  0  0  0  0  4  1  0  0  0  0  0  0  0  0  0
  0  0  0  0  2  3  0  0  0  9 44 79 99 97 73 35  0  0  0  2  0  0  0  0  0  0  0  0
  0  0  0  1  0  0 24 93162217249255255255255245199 99  2  0  2  0  0  0  0  0  0  0
  0  0  1  0 30140227255255255255255248253255255255255192 36  0  2  0  0  0  0  0  0
  0  1  0 57223255255252207154101 60 40 51 95185255250255224 30  1  2  0  0  0  0  0
  0  1  0 77218231162 60  6  0  0  0  0  0  0  0110249247255144  0  3  0  0  0  0  0
  0  0  0  2 32 34  0  0  0  3  4  3  2  2  5  1  4234254253177  0  2  0  0  0  0  0
  0  0  0  0  0  0  1  3  1  0  0  0  1  1  5  2 97255248255112  0  4  0  0  0  0  0
  0  0  0  0  2  2  0  0  0  3  4  2  0  0  0  0196255253240 21  0  1  0  0  0  0  0
  0  0  0  0  0  0  0  0  0  0  0  0  6 23 30114254249255143  0  6  2  0  0  0  0  0
  0  0  0  0  0  0  1  1  0 68137189222239247255252251254141 35  0  0  3  0  0  0  0
  0  0  0  0  0  0  2  015125525525525525425325525525525025524516

## Carga de datos

### Carga de conjunto de entrenamiento

In [ ]:
df = pd.read_csv("digits_train_sample.csv")

In [ ]:
df

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5995,5,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5996,8,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5997,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Análisis Exploratorio

In [ ]:
df["label"].value_counts(True)

7    0.112833
1    0.111833
8    0.104667
0    0.100500
9    0.099000
6    0.098500
3    0.097167
2    0.094500
4    0.091000
5    0.090000
Name: label, dtype: float64

In [ ]:
df.sample(5)

,label,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
2450,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
897,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
5590,7,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2201,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2681,6,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Preparación de datos

In [ ]:
X = df[[x for x in df.columns if x != "label"]]
y = df[["label"]]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, train_size=.7)

In [ ]:
sc_x = MinMaxScaler()

In [ ]:
Xs = sc_x.fit_transform(X_train)

## Modelado

### LDA

#### Entrenamiento

In [ ]:
y_train.values.ravel()

array([8, 0, 4, ..., 6, 6, 4])

In [ ]:
lda = LinearDiscriminantAnalysis()
lda.fit(Xs, y_train.values.ravel())
ls_scores = cross_val_score(estimator=lda, scoring="accuracy", X=Xs, y=y_train, cv=4, n_jobs=-1)
np.mean(ls_scores), np.std(ls_scores)

(0.8173809523809523, 0.009650937237862502)

### Support Vector Machine

#### Entrenamiento

In [ ]:
svm = SVC()
svm.fit(X_train, y_train.values.ravel())
ls_scores = cross_val_score(estimator=svm, scoring="accuracy", X=X_train, y=y_train, cv=4, n_jobs=-1)
np.mean(ls_scores), np.std(ls_scores)

(0.9480952380952381, 0.0054084841388574246)

#### Hyperparametrización

In [ ]:
svm?

In [ ]:
param_grid = {"C": [x for x in range(10)],
              "kernel": ['linear', 'poly', 'rbf', 'sigmoid']}

In [ ]:
n_hyper = np.prod([x for x in map(len, param_grid.values())])

In [ ]:
n_hyper

40

In [ ]:
%%time
model = SVC()
clf = GridSearchCV(model, param_grid, cv=4, error_score=-1000, n_jobs=-1, scoring="accuracy", verbose=5)
clf.fit(Xs, y_train.values.ravel())

Fitting 4 folds for each of 40 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 8 concurrent workers.
[Parallel(n_jobs=-1)]: Done   2 tasks      | elapsed:    0.1s
[Parallel(n_jobs=-1)]: Done  72 tasks      | elapsed:  1.7min
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:  3.6min finished


CPU times: user 7.29 s, sys: 43.7 ms, total: 7.34 s
Wall time: 3min 43s


GridSearchCV(cv=4, error_score=-1000,
             estimator=SVC(C=1.0, break_ties=False, cache_size=200,
                           class_weight=None, coef0=0.0,
                           decision_function_shape='ovr', degree=3,
                           gamma='scale', kernel='rbf', max_iter=-1,
                           probability=False, random_state=None, shrinking=True,
                           tol=0.001, verbose=False),
             iid='deprecated', n_jobs=-1,
             param_grid={'C': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9],
                         'kernel': ['linear', 'poly', 'rbf', 'sigmoid']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring='accuracy', verbose=5)

In [ ]:
print("Best score: " + str(clf.best_score_))
print("Best estimator: " + str(clf.best_estimator_))

Best score: 0.9545238095238096
Best estimator: SVC(C=4, break_ties=False, cache_size=200, class_weight=None, coef0=0.0,
    decision_function_shape='ovr', degree=3, gamma='scale', kernel='rbf',
    max_iter=-1, probability=False, random_state=None, shrinking=True,
    tol=0.001, verbose=False)


### Validación del modelo

#### Análisis de resultados

In [ ]:
y_pred = svm.predict(X_train)

In [ ]:
confusion_matrix(y_true=y_train, y_pred=y_pred)

array([[412,   0,   0,   0,   0,   0,   1,   0,   2,   0],
       [  0, 484,   2,   1,   0,   0,   0,   0,   1,   0],
       [  1,   1, 381,   0,   1,   0,   1,   3,   0,   0],
       [  1,   0,   2, 408,   0,   1,   0,   0,   1,   1],
       [  0,   0,   1,   0, 377,   0,   0,   0,   0,   0],
       [  0,   0,   1,   2,   0, 368,   0,   0,   1,   0],
       [  0,   1,   0,   0,   0,   2, 403,   0,   0,   0],
       [  0,   0,   0,   1,   2,   0,   0, 487,   0,   2],
       [  0,   3,   0,   2,   1,   1,   0,   0, 415,   2],
       [  1,   1,   0,   1,   2,   0,   0,   2,   1, 415]])

#### Pruebas del modelo en datos no vistos

In [ ]:
df_val = pd.read_csv("digits_test_sample.csv")

In [ ]:
X_val = df_val[[x for x in df_val.columns if x != "label"]]
y_val = df_val[["label"]]

In [ ]:
svm.score(X_val, y_val)

0.947

In [ ]:
X_val

,1x1,1x2,1x3,1x4,1x5,1x6,1x7,1x8,1x9,1x10,...,28x19,28x20,28x21,28x22,28x23,28x24,28x25,28x26,28x27,28x28
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
996,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
997,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
998,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
accuracy_score(y_pred=svm.predict(X_val), y_true=y_val)

0.947

#### Pruebas del modelo en datos reales

In [ ]:
data = flatten_data(get_data(5))

In [ ]:
svm.predict(data)

array([5])

In [ ]:
print_data(get_data(5))

  0  0  0  0  0  0  0  0  0  0  0  0  0  0  2  3  4  4  4  4  2  0  0  0  0  0  0  0
  0  0  0  0  1  0 26 79 64 40 26 16  7  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  1  0 11220255255255255244230213194171148124 99 73 27  0  0  0  0  0  0  0
  0  0  0  0  2  4188252250219216232244255255255255255255255223 26  0  2  0  0  0  0
  0  0  0  0  4  0 81255255120  0 12 18 30 45 63 82102121139 96  8  0  1  0  0  0  0
  0  0  0  0  2  0 33246255177  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  1  0 25240255190  1  3  1  2  2  3  4  4  4  4  3  0  0  0  0  0  0  0
  0  0  0  0  2  0 37249255183  1  5  4  4  4  4  3  2  0  0  0  0  0  0  0  0  0  0
  0  0  0  0  2  0 49254255174  0  0  0  0  0  0  0  0  0  4  1  0  0  0  0  0  0  0
  0  0  0  0  3  0 64255255148  3 44 76101110100 74 36  0  0  0  3  0  0  0  0  0  0
  0  0  0  0  3  0 79255252217211255255255255255255250199111 14  0  2  1  0  0  0  0
  0  0  0  0  4  0 94252246255255243204172161174208254255255219 8